In [ ]:
# Imports
from ttt.env.state import TTTEnv
from ttt.agents.random import RandomAgent
from ttt.mcts.base import GenericMCTS, MCTSConfig
from ttt.eval.alphazero_factory import alphazero_agent_factory
from ttt.mcts.alphazero_strategy import infer
import torch

In [210]:
# Setup Blunder Board
env = TTTEnv(seed=15)
env.step(0)
env.step(1)
env.step(5)
env.step(7)
env.step(4)
print(env.board)


[[ 1 -1  0]
 [ 0  1  1]
 [ 0 -1  0]]


In [203]:
# Load agents
random = RandomAgent()
az_factory = alphazero_agent_factory(checkpoint_path="ttt/runs/run_20251025_221904/checkpoints/alphazero_epoch_25.pt", 
                             mcts_simulations=100,
                             temperature=0.0)
az = az_factory(**{})

c:\Users\occoo\UTTT\ttt\eval\alphazero_factory.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


In [221]:
az.set_temperature(0.7)
mcts = GenericMCTS(az.strategy, az.mcts_config)
best_action, actions, action_probs = mcts.search(env)
print(best_action)
print(actions)
print(action_probs)
print(type(action_probs))

8
[0.         0.         0.07331613 0.22869677 0.         0.
 0.04760562 0.         0.65038147]
[0.         0.         0.01037973 0.18458545 0.         0.
 0.01548662 0.         0.7895482 ]
<class 'numpy.ndarray'>


In [213]:
obs = env._encode()  # Shape: (5, 3, 3)
legal_mask = env.legal_actions_mask()  # Shape: (3, 3)

strategy = az.strategy

# Convert to tensors and add batch dimension
obs_tensor = torch.from_numpy(obs).unsqueeze(0).float().to(strategy.device)
legal_mask_tensor = torch.from_numpy(legal_mask.flatten()).unsqueeze(0).bool().to(strategy.device)

net = az.strategy.network
policy, value = net.predict_priors_value(obs_tensor, legal_mask_tensor)

In [215]:
print(policy)
print(value)

tensor([[0.0000, 0.0000, 0.0104, 0.1846, 0.0000, 0.0000, 0.0155, 0.0000, 0.7895]])
tensor([-0.3396])
